# Collection and summarizing facts of Trump-Zelenskyy meeting

In [31]:
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
from pydantic import BaseModel, Field
from dataclasses import dataclass
from pydantic_ai import Agent, ModelRetry, RunContext
from typing import Optional
import os
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openrouter import OpenRouterProvider

## Filtering incoming tweets

In [33]:
class TopicFilter(BaseModel):
    """Whether this is related to Trump/Zelenskyy meeting"""
    topic_match: bool = Field(description = "Whether this news match the topic (Yes or No)")
    explanation: Optional[str] = Field(description = "explanation after of Yes/No answer")

In [34]:
@dataclass
class Deps:
    """Dependencies"""

In [35]:
model = OpenAIModel(
    'x-ai/grok-4',
    provider=OpenRouterProvider(api_key=os.getenv("OPENROUTER_API_KEY")),
)


topic_filter_agent = Agent[Deps, TopicFilter](
    model,
    output_type=TopicFilter,
    retries=3,
    system_prompt=(
        '''
            ## Role
            You are an AI classifier specialized in analyzing news tweets.
            Your task is to determine if a given tweet from one of the following mass media accounts—@BBCBreaking, @Reuters, @AP, @Axios, or @FoxNews—is related to concrete, actionable outcomes of the meeting between U.S. President Donald Trump and Ukrainian President Volodymyr Zelenskyy, scheduled for August 18, 2025, at the White House in Washington, D.C., focusing on ending Russia’s war in Ukraine.
            ## Input
            You will receive the text of a single tweet as input. Assume it is from one of the listed accounts.
            ## Output
            Respond with exactly one of the following:
                - "Yes" if the tweet describes specific, actionable outcomes of the Trump-Zelenskyy meeting, such as concrete agreements, measurable commitments, or real policy changes (e.g., signed deals, ceasefire terms, funding pledges, or troop deployments). Use semantic and contextual analysis to confirm relevance, even if the date, location, or terms like "meeting" are omitted.
            - "No" if the tweet is unrelated or only mentions ceremonial aspects (e.g., photo ops, handshakes, or general statements without actionable results).
            Provide a brief 1-2 sentence explanation after your Yes/No answer, justifying your decision based on key elements in the tweet. Do not output anything else.
        '''
    ),
)

# Test tweets filtering by topic

## Ceremonial Tweets (Should be "No"):

- @AP: Presidents Trump and Zelenskyy exchange warm handshakes at White House arrival for Ukraine talks.

- @Reuters: Trump welcomes Zelenskyy with a photo op in the Oval Office ahead of discussions on peace.

- @BBCBreaking: Leaders pose for cameras at the start of the White House summit on Ukraine.

- @Axios: Ceremonial dinner hosted by Trump for Zelenskyy and European allies post-meeting greetings.

- @FoxNews: Trump and Zelenskyy share opening remarks praising mutual respect at White House event.

## Actionable Tweets (Should be "Yes"):

- @AP: Trump and Zelenskyy agree to $10B US aid package for Ukraine reconstruction in White House deal.

- @Reuters: Leaders commit to joint monitoring of Ukraine ceasefire with NATO involvement post-summit.

- @BBCBreaking: White House meeting yields policy shift: US to provide advanced weaponry to Ukraine forces.

- @Axios: Trump-Zelenskyy pact includes measurable troop withdrawal timeline from key Ukraine regions.

- @FoxNews: Agreement reached on economic sanctions relief in exchange for Russia-Ukraine peace terms.

In [36]:
import json

tests_json = '''
[
  {
    "headline": "Presidents Trump and Zelenskyy exchange warm handshakes at White House arrival for Ukraine talks.",
    "is_relevant": false
  },
  {
    "headline": "Trump welcomes Zelenskyy with a photo op in the Oval Office ahead of discussions on peace.",
    "is_relevant": false
  },
  {
    "headline": "Leaders pose for cameras at the start of the White House summit on Ukraine.",
    "is_relevant": false
  },
  {
    "headline": "Ceremonial dinner hosted by Trump for Zelenskyy and European allies post-meeting greetings.",
    "is_relevant": false
  },
  {
    "headline": "Trump and Zelenskyy share opening remarks praising mutual respect at White House event.",
    "is_relevant": false
  },
  {
    "headline": "Trump and Zelenskyy agree to $10B US aid package for Ukraine reconstruction in White House deal.",
    "is_relevant": true
  },
  {
    "headline": "Leaders commit to joint monitoring of Ukraine ceasefire with NATO involvement post-summit.",
    "is_relevant": true
  },
  {
    "headline": "White House meeting yields policy shift: US to provide advanced weaponry to Ukraine forces.",
    "is_relevant": true
  },
  {
    "headline": "Trump-Zelenskyy pact includes measurable troop withdrawal timeline from key Ukraine regions.",
    "is_relevant": true
  },
  {
    "headline": "Agreement reached on economic sanctions relief in exchange for Russia-Ukraine peace terms.",
    "is_relevant": true
  }
]
'''

test_cases = json.loads(tests_json)

In [44]:
import asyncio

max_concurrent = 5
# Semaphore to limit concurrent requests
semaphore = asyncio.Semaphore(max_concurrent)

async def process_single_case_with_limit(test_case: str, index: int):
    """Process a single test case with concurrency limit"""
    async with semaphore:
        try:
            result = await topic_filter_agent.run(test_case['headline'])
            return {
                'promt': test_case['headline'],
                'result': result,
                'expected': test_case['is_relevant'],
                'success': True,
                'error': None
            }
        except Exception as e:
            return {
                'index': index,
                'promt': test_case['headline'],
                'result': None,
                'success': False,
                'error': str(e)
            }


tasks = [
    process_single_case_with_limit(test_case, index) 
    for index, test_case in enumerate(test_cases)
]
results = await asyncio.gather(*tasks)

In [50]:
print("Testing News Filtering Agent")
print("=" * 50)

for result in results:
    try:
        status = "✓ PASS" if result['expected'] == result['result'].output.topic_match else "✗ FAIL"
        print(f"{status} | '{result['promt']}' -> {result['result'].output.topic_match} (expected: {result['expected']})")
    except Exception as e:
        print(f"✗ ERROR | '{result['promt']}' -> Error: {e}")

print("\n" + "=" * 50)
print("Test completed!")

Testing News Filtering Agent
✓ PASS | 'Presidents Trump and Zelenskyy exchange warm handshakes at White House arrival for Ukraine talks.' -> False (expected: False)
✓ PASS | 'Trump welcomes Zelenskyy with a photo op in the Oval Office ahead of discussions on peace.' -> False (expected: False)
✓ PASS | 'Leaders pose for cameras at the start of the White House summit on Ukraine.' -> False (expected: False)
✓ PASS | 'Ceremonial dinner hosted by Trump for Zelenskyy and European allies post-meeting greetings.' -> False (expected: False)
✓ PASS | 'Trump and Zelenskyy share opening remarks praising mutual respect at White House event.' -> False (expected: False)
✓ PASS | 'Trump and Zelenskyy agree to $10B US aid package for Ukraine reconstruction in White House deal.' -> True (expected: True)
✓ PASS | 'Leaders commit to joint monitoring of Ukraine ceasefire with NATO involvement post-summit.' -> True (expected: True)
✓ PASS | 'White House meeting yields policy shift: US to provide advanced we

## Prevent duplications

In [62]:
class DuplicateCheckResult(BaseModel):
    """Result of duplicate check - returns only a boolean value"""
    is_duplicate: bool = Field(description="True if the news is duplicate, False if it's unique")


class NewsDatabase:
    """Simple database to store existing news items"""
    
    def __init__(self, existing_news: List[str]):
        self.existing_news = existing_news
    
    def get_existing_news(self) -> List[str]:
        return self.existing_news


In [63]:
# Create the agent with explicit output type
duplicate_detector_agent = Agent[NewsDatabase, DuplicateCheckResult](
    model,
    deps_type=NewsDatabase,
    output_type=DuplicateCheckResult,
    system_prompt=(
        "You are a news duplicate detection expert. Your job is to determine if a new news tweet "
        "conveys the same information as any existing news in the database, even if worded differently. "
        "Focus on the core information and facts, not the exact wording. "
        "Return True if the news is substantially similar to existing news, False if it's unique."
    ),
)

In [64]:
@duplicate_detector_agent.tool
async def check_against_existing_news(
    ctx: RunContext[NewsDatabase], 
    new_news: str
) -> str:
    """Compare the new news against all existing news items in the database."""
    existing_news = ctx.deps.get_existing_news()
    
    if not existing_news:
        return "No existing news found in database."
    
    news_list = "\n".join([f"{i+1}. {news}" for i, news in enumerate(existing_news)])
    
    return f"""
    New news to check: "{new_news}"
    
    Existing news in database:
    {news_list}
    
    Please analyze if the new news conveys substantially the same information 
    as any of the existing news items, considering:
    - Core facts and events
    - Key people involved
    - Main outcomes or decisions
    - Geographic locations
    - Time periods mentioned
    
    Ignore differences in:
    - Exact wording or phrasing
    - Writing style
    - Minor details that don't change the core message
    """

In [69]:
async def is_duplicate_news(new_news: str, existing_news: List[str]) -> bool:
    """
    Async version of duplicate detection for better performance.
    
    Args:
        new_news: The new news tweet to check
        existing_news: List of existing news tweets
    
    Returns:
        bool: True if duplicate, False if unique
    """
    news_db = NewsDatabase(existing_news)
    
    result = await duplicate_detector_agent.run(
        f"Is this news duplicate? Check: '{new_news}'",
        deps=news_db
    )
    
    return result.output.is_duplicate

## Test prevent duplications

In [70]:
input_outcomes = [
    "Trump and Zelenskyy agree to a $10B US aid package for Ukraine reconstruction, to be disbursed over two years.",
    "Leaders commit to establishing a NATO-led ceasefire monitoring mission in eastern Ukraine by September 2025.",
    "US agrees to supply Ukraine with advanced air defense systems to protect key cities, effective immediately.",
    "No agreement reached on Russian troop withdrawal from occupied Ukrainian territories."
]

# Test cases
test_cases = [
    ("Russia will not give up captured land", True),  # Should be duplicate of last item
    ("New $5B trade deal announced between US and Canada", False),  # Should be unique
    ("Putin refuses to withdraw troops from Ukrainian regions", True),  # Should be duplicate
    ("Ukraine to receive air defense systems from America", True),  # Should be duplicate
    ("NATO to monitor ceasefire in eastern Ukraine from September", True),  # Should be duplicate
    ("Biden announces new climate change initiative", False),  # Should be unique
]

In [71]:
async def process_single_case_with_limit(test_case: str, expected: bool, index: int):
    """Process a single test case with concurrency limit"""
    async with semaphore:
        try:
            result = await is_duplicate_news(test_case, input_outcomes)
            return {
                'tweet': test_case,
                'result': result,
                'expected': expected,
                'success': True,
            }
        except Exception as e:
            return {
                'index': index,
                'tweet': test_case,
                'result': None,
                'success': False,
                'error': str(e)
            }


tasks = [
    process_single_case_with_limit(test_case, expected, index) 
    for index, (test_case, expected) in enumerate(test_cases)
]
results = await asyncio.gather(*tasks)

In [75]:
print("Testing News Duplicate filtering Agent")
print("=" * 50)

for result in results:
    try:
        status = "✓ PASS" if result['expected'] == result['result'] else "✗ FAIL"
        print(f"{status} | '{result['tweet']}' -> {result['result']} (expected: {result['expected']})")
    except Exception as e:
        print(f"✗ ERROR | '{result['tweet']}' -> Error: {e}")

print("\n" + "=" * 50)
print("Test completed!")

Testing News Duplicate filtering Agent
✓ PASS | 'Russia will not give up captured land' -> True (expected: True)
✓ PASS | 'New $5B trade deal announced between US and Canada' -> False (expected: False)
✓ PASS | 'Putin refuses to withdraw troops from Ukrainian regions' -> True (expected: True)
✓ PASS | 'Ukraine to receive air defense systems from America' -> True (expected: True)
✓ PASS | 'NATO to monitor ceasefire in eastern Ukraine from September' -> True (expected: True)
✓ PASS | 'Biden announces new climate change initiative' -> False (expected: False)

Test completed!


## Summarizing the facts

In [76]:
from typing import List

class OutcomeAnalysis(BaseModel):
    description: str = Field(..., min_length=1, description="Description of the actionable outcome from the meeting")
    significance: str = Field(..., min_length=1, description="Brief analysis of the outcome's significance (1-2 sentences)")
    impact_score: int = Field(..., ge=1, le=10, description="Impact score from 1 to 10 for the outcome's effect on peace or ceasefire")

class MeetingAnalysis(BaseModel):
    outcomes: List[OutcomeAnalysis] = Field(..., min_items=1, description="List of analyzed outcomes from the Trump-Zelenskyy meeting")
    overall_score: int = Field(..., ge=1, le=10, description="Overall score from 1 to 10 for likelihood of peace deal or ceasefire")
    overall_explanation: str = Field(..., min_length=1, description="Brief explanation for the overall score (1-2 sentences)")

In [77]:
model = OpenAIModel(
    'x-ai/grok-4',
    provider=OpenRouterProvider(api_key=os.getenv("OPENROUTER_API_KEY")),
)


geo_expert_agent = Agent[Deps, MeetingAnalysis](
    model,
    output_type=MeetingAnalysis,
    retries=3,
    system_prompt=(
        '''
            ## Role
            You are a geopolitical expert with comprehensive knowledge of the Russia-Ukraine conflict, including historical context, key stakeholders, ongoing military dynamics, diplomatic efforts, and potential pathways to resolution.
            
            ## Input
            You will be given a list of actionable outcomes from the Trump-Zelenskyy meeting (e.g., concrete agreements, commitments, or policy changes). Assume the meeting occurred on August 18, 2025, at the White House, focusing on ending Russia's war in Ukraine.
            
            ## Task
            For each actionable outcome in the list:
            - Analyze its significance in the context of the Russia-Ukraine conflict.
            - Rate its individual impact on the likelihood of achieving a peace deal or temporary ceasefire (1-10 scale, where 1 means it has negligible or negative effect, and 10 means it directly resolves major issues).
            
            Then, provide an overall score (1-10) assessing how likely the combined outcomes are to lead to a peace deal or at least a temporary ceasefire within a defined timeline (e.g., weeks or months). Use this scale:
            - 1: The outcomes make no difference or even reduce the chances of ceasefire/peace.
            - 5: Moderate progress toward dialogue, but no binding commitments or timelines.
            - 10: The end of the war is effectively announced, with a set date for implementation.
            
            ## Output
            Structure your response as follows:
            - For each item: [Item description] - Significance: [Brief analysis, 1-2 sentences]. Impact Score: [1-10].
            - Overall Score: [1-10] - Explanation: [1-2 sentences justifying the score based on the combined outcomes].
            Do not output anything else.
        '''
    ),
)

In [78]:
input_outcomes = [
    "Trump and Zelenskyy agree to a $10B US aid package for Ukraine reconstruction, to be disbursed over two years.",
    "Leaders commit to establishing a NATO-led ceasefire monitoring mission in eastern Ukraine by September 2025.",
    "US agrees to supply Ukraine with advanced air defense systems to protect key cities, effective immediately.",
    "No agreement reached on Russian troop withdrawal from occupied Ukrainian territories."
]

## Test summarizing the facts

In [79]:
result = await geo_expert_agent.run(input_outcomes)

In [85]:
print(json.dumps(result.output.model_dump(), indent=2, ensure_ascii=False))

{
  "outcomes": [
    {
      "description": "Trump and Zelenskyy agree to a $10B US aid package for Ukraine reconstruction, to be disbursed over two years.",
      "significance": "This aid package demonstrates continued US support for Ukraine's long-term stability and reconstruction, potentially incentivizing peace by focusing on post-conflict recovery, though it does not directly address ongoing military actions.",
      "impact_score": 5
    },
    {
      "description": "Leaders commit to establishing a NATO-led ceasefire monitoring mission in eastern Ukraine by September 2025.",
      "significance": "Establishing a NATO-led monitoring mission represents a tangible step toward de-escalation and oversight of a potential ceasefire, which could build trust and facilitate negotiations by involving international actors in the conflict zone.",
      "impact_score": 8
    },
    {
      "description": "US agrees to supply Ukraine with advanced air defense systems to protect key cities, 